In [5]:
import mysql.connector

In [6]:
conn=mysql.connector.connect(host='localhost',port=int(3306),user='root',passwd='Troyrules@2',db='mysql_v2')
cursor = conn.cursor()

In [7]:
insert_query = "INSERT INTO mysql_v2.new_table (col_1) VALUES (10), (13)"
cursor.execute(insert_query)

ProgrammingError: 1146 (42S02): Table 'mysql_v2.new_table' doesn't exist

In [8]:
query = "SELECT * FROM mysql_v2.new_table"

cursor.execute(query)
for x in cursor:
    print(x)

ProgrammingError: 1146 (42S02): Table 'mysql_v2.new_table' doesn't exist

In [6]:
cursor.close()
conn.close()

In [15]:
import requests

api_key = "6QRI1R8DTP46EKC9QJBBWU898A4UN7KJUC" #api held in personal etherscan account --> handles 5 req/sec, as demonstrated below
for i in range(1, 10):
    response = requests.get("https://api.etherscan.io/api?module=gastracker&action=gasoracle&apikey={}".format(api_key))
    print(response.json())


{'status': '1', 'message': 'OK', 'result': {'LastBlock': '14515902', 'SafeGasPrice': '40', 'ProposeGasPrice': '41', 'FastGasPrice': '42', 'suggestBaseFee': '39.725183686', 'gasUsedRatio': '0.336943428238234,0.638848270887272,0.915901151675016,0.217773866551632,0.213430813982744'}}
{'status': '1', 'message': 'OK', 'result': {'LastBlock': '14515902', 'SafeGasPrice': '40', 'ProposeGasPrice': '41', 'FastGasPrice': '42', 'suggestBaseFee': '39.725183686', 'gasUsedRatio': '0.336943428238234,0.638848270887272,0.915901151675016,0.217773866551632,0.213430813982744'}}
{'status': '1', 'message': 'OK', 'result': {'LastBlock': '14515902', 'SafeGasPrice': '40', 'ProposeGasPrice': '41', 'FastGasPrice': '42', 'suggestBaseFee': '39.725183686', 'gasUsedRatio': '0.336943428238234,0.638848270887272,0.915901151675016,0.217773866551632,0.213430813982744'}}
{'status': '1', 'message': 'OK', 'result': {'LastBlock': '14515902', 'SafeGasPrice': '40', 'ProposeGasPrice': '41', 'FastGasPrice': '42', 'suggestBaseFee'

In [10]:
import json
api_key = "6QRI1R8DTP46EKC9QJBBWU898A4UN7KJUC" #api held in personal etherscan account 
response = requests.get("https://api.etherscan.io/api?module=gastracker&action=gasoracle&apikey={}".format(api_key))

var = response.json()
#extracting SafeGasPrice(slow), FastGasPrice (fast), ProposeGasPrice (average)
safe_gas = var["result"]["SafeGasPrice"]
fast_gas = var["result"]["FastGasPrice"]
prop_gas = var["result"]["ProposeGasPrice"]


In [14]:
#create query to load data
drop = "DROP TABLE gas_data"
cursor.execute(drop)
create_query = "CREATE TABLE gas_data (time BIGINT, safe_gas BIGINT, fast_gas BIGINT, propose_gas BIGINT)"
cursor.execute(create_query)

print(cursor.description)


None


In [13]:
#gathering UNIX time
import time
now = int( time.time() )
next_time = now

while(1): 
    now = int( time.time())
    #include to insert new data every second
    if next_time == now:
        next_time = next_time + 1
        api_key = "6QRI1R8DTP46EKC9QJBBWU898A4UN7KJUC" #api held in personal etherscan account 
        response = requests.get("https://api.etherscan.io/api?module=gastracker&action=gasoracle&apikey={}".format(api_key))

        var = response.json()
        if var["message"] == "OK":
            #extracting SafeGasPrice(slow), FastGasPrice (fast), ProposeGasPrice (average)
            safe_gas = var["result"]["SafeGasPrice"]
            fast_gas = var["result"]["FastGasPrice"]
            prop_gas = var["result"]["ProposeGasPrice"]

            #loading time, safe_gas, fast_gas, and propse_gas data into new query
            insert_query = "INSERT INTO gas_data (time, safe_gas, fast_gas, propose_gas) VALUES (%s, %s, %s, %s)"
            val = (now, safe_gas, fast_gas, prop_gas)
            cursor.execute(insert_query, val)
            conn.commit()



KeyboardInterrupt: 

In [117]:
from flask import Flask
from flask_restful import Resource, Api, reqparse

#API initialization
app = Flask(__name__)
api = Api(app)

if __name__ == "__main__":
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [107]:
var = response.json()
recent_data = "SELECT * FROM gas_data WHERE time = (SELECT MAX(time) FROM gas_data)"
cursor.execute(recent_data)

#fetching most recent data
print(cursor.fetchone())

(1648876222, 30, 30, 30)
